Both wm1 and m7 has been filtered at high frequency for wind at 230+/- 30 aggregated at 5 mins

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from wind_tools import *  
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib
#matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt
sns.set()
import plotly.express as px

matplotlib.rcParams['figure.figsize'] = (16,10)

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize

## Load data

In [ ]:
proc_dir = Path("2020_data/data_20200724_final_field_install/processed")

# default is 2 mins aggregated data
wm1_path = proc_dir / "eddypro_WM1_filtered_full_output_2020-08-07T171756_adv.csv"
m7_path = proc_dir / "eddypro_TRS_M507_filtered_230_30_full_output_2020-08-07T122855_exp.csv"



cache_dir = Path("data_cache")
start_date = '2020-07-24 13:00:00'
end_date = '2020-08-07 09:00:00'

wind_cols = ['u_unrot','v_unrot', 'w_unrot', 'wind_speed', 'wind_dir', 'Tau']
wind_comp_rename = {'u_unrot': 'u', 'v_unrot': 'v', 'w_unrot': 'w'}

In [ ]:
wm1, m7= map(
    lambda p: load_ep_cache(p, cache_dir= cache_dir)
    .loc[start_date:end_date]
    .loc[:,wind_cols]
    .rename(columns=wind_comp_rename),
    [wm1_path, m7_path])

In [ ]:
wm1.describe()

,u,v,w,wind_speed,wind_dir,Tau
count,743.000000,743.000000,743.000000,743.000000,743.000000,743.000000
mean,-0.517188,-0.524190,-0.037841,1.685539,139.091328,-0.064023
std,0.386897,1.720484,0.130332,0.909007,107.407128,0.126779
min,-1.501140,-3.832720,-0.545024,0.078846,3.449320,-0.631983
25%,-0.834501,-2.125830,-0.122343,0.875562,28.999150,-0.130880
50%,-0.475743,-0.273689,-0.017784,1.690980,236.165000,-0.045772
75%,-0.179550,0.865159,0.049331,2.402770,241.143500,0.003154
max,0.157657,3.246810,0.381188,4.107640,358.528000,0.484350


In [ ]:
m7.describe()

,u,v,w,wind_speed,wind_dir,Tau
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,1.860107,0.337184,0.103020,1.997371,56.137663,-0.077071
std,0.974987,0.126241,0.183205,0.737355,59.390475,0.157672
min,-2.144150,-0.305491,-0.594317,0.116625,1.772560,-0.614078
25%,1.550875,0.258949,-0.028386,1.574158,37.105525,-0.183433
50%,2.065565,0.345121,0.088177,2.104115,39.120650,-0.089294
75%,2.455600,0.423061,0.204474,2.485350,40.574450,0.018820
max,3.558320,0.683249,0.679841,3.628410,337.866000,0.361436


In [ ]:
wm1.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-06 23:15:00', '2020-08-06 23:20:00',
               '2020-08-06 23:25:00', '2020-08-06 23:30:00',
               '2020-08-06 23:35:00', '2020-08-06 23:40:00',
               '2020-08-06 23:45:00', '2020-08-06 23:50:00',
               '2020-08-06 23:55:00', '2020-08-07 00:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3876, freq=None)

In [ ]:
plot_components([wm1, m7], ['wind_speed'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:title={'center':'wind_speed'}, xlabel='date_time'>],
      dtype=object)

In [ ]:
wm1r = rotate_wind_ang(wm1, 310)
wm1r = add_hor_wind_speed(wm1r)